In [ ]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import time

# CSV 파일에서 titleId 값을 가져오기
title_df = pd.read_csv("naver_listly_updated.csv")  # 네이버 웹툰 ID가 들어 있는 CSV
id_list = title_df['ID'].astype(str).tolist()  # 문자열로 변환
id_list = id_list[:10]  # 10개만 테스트용으로 가져오기

# URL 생성
base_url = 'https://comic.naver.com/webtoon/list?titleId='
urls = [f"{base_url}{num}" for num in id_list]

# WebDriver 설정
options = Options()
options.add_argument('headless')
options.add_argument('window-size=1920x1080')
options.add_argument("disable-gpu")
driver = webdriver.Chrome(options=options)

# 데이터 저장 리스트
data = []

# URL 순회하며 정보 수집
for url in urls:
    driver.get(url)
    time.sleep(2)

    soup = BeautifulSoup(driver.page_source, "html.parser")

    # 1. a 태그 모두 찾기 (작가명이 여러 명일 수 있음)
    author_links = soup.find_all('a', class_='ContentMetaInfo__link--xTtO6')

    # 2. 모든 작가명을 추출하여 쉼표로 구분
    author_names = [author_link.get_text(strip=True) for author_link in author_links]
    author_names_str = ",".join(author_names) if author_names else "작가 정보를 찾을 수 없음"

    # 저장
    data.append({
        'Authors': author_names_str
    })

    print("현재 작가들:", author_names_str)
    time.sleep(5)

# 드라이버 종료
driver.quit()

# DataFrame 생성 및 출력
df = pd.DataFrame(data)
print(df)
df.to_csv("webtoon_authors_test.csv", index=False, encoding='utf-8-sig')
